# Context Tree Weighting Prediction

In [1]:
# CTW Prediction
import json
import urllib.request
import os
os.chdir("/home/tales/dev/master/mdc_analysis/")
print("working dir", os.getcwd())

from src.ml.ctw import MyCTW
from src.dao import csv_dao
from src.entity.stop_region import StopRegionGroup, sr_row_to_stop_region
from src.exceptions import exceptions
from src.utils.others import remove_list_elements
from src.exceptions.exceptions import NoCategoryMatched

from src.experiments.ctw_eval import test_ctw

from src.taxonomy.category_mapping import CategoryMapper

from src.exceptions.exceptions import NoCategoryMatched, NotValidTypes

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
pd.set_option('display.float_format', lambda x: '%.3f' % x)

working dir /home/tales/dev/master/mdc_analysis


## Loading Data

In [2]:
def get_users_tags_sequence():
    url = "http://127.0.0.1:5000/stop_regions_group"
    response = urllib.request.urlopen(url)
    return json.loads(response.read())

def load_users_tags_sequence():
    with open('outputs/users_tags_sequence.json') as json_file:
        return json.load(json_file)
    
users_tags_sequence = load_users_tags_sequence()

In [3]:
def clean_sequence(sequence):
    new_sequence = []
    
    for tags in sequence:
        if "parking" in tags:
            continue
        else:
            new_sequence.append(tags)

    return new_sequence



In [4]:
categ_mapper = CategoryMapper()

In [5]:
users_categ_sequence = {}

for user_id in users_tags_sequence:
    print("user_id:", user_id)
    
    users_tags_sequence[user_id] = clean_sequence(users_tags_sequence[user_id])
    
    if len( remove_list_elements(users_tags_sequence[user_id], elements=[[]]) ) < 8:
        continue 
    
    categ_sequence = []

    for tags in users_tags_sequence[user_id]:
        if tags == ["WORK"] or tags == ["HOME"]:
            categ_sequence.append(tags)

        else:
            try:
#                 print("tags:", tags)
                categ = categ_mapper.map_categ(tags, method="most_specific")
                categ_sequence.append([categ])

            except NotValidTypes:
                pass
            
            except NoCategoryMatched:
                categ_sequence.append(["NoCategoryMatched"]) 

    users_categ_sequence[user_id] = categ_sequence
    

user_id: 6189
user_id: 5936
user_id: 6087
user_id: 5973
user_id: 6085
user_id: 6074
user_id: 6012
user_id: 5982
user_id: 5948
user_id: 5974
user_id: 6090
user_id: 6199
user_id: 6068
user_id: 6024
user_id: 5976
user_id: 6094
user_id: 5941
user_id: 5995
user_id: 5962
user_id: 6093
user_id: 6033
user_id: 6079
user_id: 6038
user_id: 6175
user_id: 6042
user_id: 5924
user_id: 6083
user_id: 6178
user_id: 5958
user_id: 6086
user_id: 6100
user_id: 5945
user_id: 5925
user_id: 5935
user_id: 6172
user_id: 5955
user_id: 6073
user_id: 5980
user_id: 6010
user_id: 5993
user_id: 6037
user_id: 5979
user_id: 5966
user_id: 5985
user_id: 5967
user_id: 5970
user_id: 6169
user_id: 6188
user_id: 6097
user_id: 6096
user_id: 6041
user_id: 6192
user_id: 6064
user_id: 6183
user_id: 5954
user_id: 5972
user_id: 6060
user_id: 6069
user_id: 5937
user_id: 6028
user_id: 5990
user_id: 6071
user_id: 5928
user_id: 5992
user_id: 6058
user_id: 6001
user_id: 5964
user_id: 5950
user_id: 5991
user_id: 6177
user_id: 5961
user_i

In [ ]:
# users_tags_sequence = {}

# users = os.listdir("outputs/stop_regions/")
# users.reverse()

# for user_id in users:
#     print("Loading user {} data".format(user_id))
#     users_tags_sequence[user_id] = StopRegionGroup(csv_dao.stop_region_sequence(user_id), 
#                                                     agglutinate_stop_regions=True).sequence_stop_region_tags()["tag"].tolist()

## Evaluation

In [15]:
skipped_users = []

depths = [2,3,4]

predict_choice_methods = ["random_choice", "most_likely"]

for user_id in users_categ_sequence.keys():
    print("user_id:", user_id)
    
    for depth in depths:
        
        for predict_choice_method in predict_choice_methods:
    
            tags_sequence = [categ[0] for categ in users_categ_sequence[user_id]]

            if len( remove_list_elements(tags_sequence, elements=[[]]) ) < 8:
                skipped_users.append(user_id)
                continue    

            try:
                test_ctw(user_id, 
                         sequence=tags_sequence, 
                         depth=depth, 
                         predict_choice_method=predict_choice_method,
                         method="same_train_and_test", 
                         input_data_version="ctw-0.0.categ_v1", 
                         save_result=True)


            except exceptions.TagsLengthNeedsToBeGreaterThanK:
                print("TagsLengthNeedsToBeGreaterThanK")



user_id: 6189
user_id: 5936
user_id: 6087
user_id: 5973
user_id: 6085
user_id: 6074
user_id: 6012
user_id: 5982
user_id: 5948
user_id: 5974
user_id: 6090
user_id: 6199
user_id: 6068
user_id: 6024
user_id: 5976
user_id: 6094
user_id: 5941
user_id: 5995
user_id: 5962
user_id: 6093
user_id: 6033
user_id: 6079
user_id: 6038
user_id: 6175
user_id: 6042
user_id: 5924
user_id: 6083
user_id: 6178
user_id: 5958
user_id: 6086
user_id: 5945
user_id: 5925
user_id: 5935
user_id: 6172
user_id: 5955
user_id: 6073
user_id: 5980
user_id: 6010
user_id: 5993
user_id: 6037
user_id: 5979
user_id: 5966
user_id: 5985
user_id: 5967
user_id: 5970
user_id: 6188
user_id: 6097
user_id: 6096
user_id: 6041
user_id: 6192
user_id: 6064
user_id: 5954
user_id: 5972
user_id: 6060
user_id: 6069
user_id: 5937
user_id: 6028
user_id: 5990
user_id: 6071
user_id: 5928
user_id: 5992
user_id: 6058
user_id: 6001
user_id: 5964
user_id: 5950
user_id: 5991
user_id: 6177
user_id: 5961
user_id: 6176
user_id: 6007
user_id: 5956
user_i

In [ ]:
skipped_users

In [ ]:
# all_users_vs_one_light_mem(users_tags_sequence, 
#                            input_data_version="markov-0.0", 
#                            is_distributive=False)

# all_users_vs_one_light_mem(users_tags_sequence, 
#                            random_dummy_mode="dummy", 
#                            input_data_version="markov-0.0-DUMMY", 
#                            is_distributive=False)


# all_users_vs_one_light_mem(users_tags_sequence, 
#                            input_data_version="markov-0.0.d", 
#                            is_distributive=True)

# all_users_vs_one_light_mem(users_tags_sequence, 
#                            random_dummy_mode="dummy", 
#                            input_data_version="markov-0.0.d-DUMMY", 
#                            is_distributive=True)


# all_users_vs_one_light_mem(users_categ_sequence, 
#                            input_data_version="markov-0.0.categ_v1", 
#                            is_distributive=False)

# all_users_vs_one_light_mem(users_categ_sequence, 
#                            random_dummy_mode="dummy", 
#                            input_data_version="markov-0.0.categ_v1-DUMMY", 
#                            is_distributive=False)